In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [331]:
N = 10000

In [349]:
def geomBrown(S0,r,sigma,T,eps):
    d = [S*np.exp((r-sigma*sigma*0.5)*T + sigma*eps[i]*np.sqrt(T)) for i,S in enumerate(S0)]
    return d

In [347]:
def geomBrownIter(S0,r,sigma,T,dt=1,KOLevel=None):
    dt=dt/365
    expiries = np.linspace(0,T,T/dt,endpoint=True)
    path = []
    #generate path i-th element
    for i,v in enumerate(expiries):
        eps = np.random.normal(0,1,N)
        if i==0:
            Si = geomBrown(np.full(N,S0,np.double),r,sigma,dt,eps)
        else:
            Si = geomBrown(path[-1],r,sigma,dt,eps)
        path.append(Si)
    return path

In [346]:
def callPayoff(S,K):
    return max(S-K,0)

def putPayoff(S,K):
    return max(K-S,0)

def vanillaPrice(S0,r,sigma,T,K,f):
    paths=geomBrownIter(S0,r,sigma,T)
    ST = paths[-1] # list simulated prices at expiry
    payoffs = [f(STi,K) for STi in ST]
    return np.mean(payoffs)*math.exp(-r*T)

def vanillaCall(S0,r,sigma,T,K):
    return vanillaPrice(S0,r,sigma,T,K,callPayOff)
    
def vanillaPut(S0,r,sigma,T,K):
    return vanillaPrice(S0,r,sigma,T,K,putPayoff)

In [336]:
vc = vanillaCall(269, 0.01, 0.43, 0.58, 270)
print(vc)

vp = vanillaPut(269, 0.01, 0.43, 0.58, 145)
print(vp)

34.5493472939
0.702011087326


In [372]:
def asianBarrierCallPayoff(SPath,K,B):
    sum=0
    for Si in SPath:
        if Si>B:
            return 0
        sum+=Si
    return max(sum/len(SPath)-K,0)
                   
def asianBarrierPutPayoff(SPath,K,B):
    sum=0
    for Si in SPath:
        if Si>B:
            return 0
        sum+=Si
        return max(K-sum/len(SPath),0)
    
def asianPrice(S0,r,sigma,T,K,f,B):
    paths=geomBrownIter(S0, r, sigma, T)
    STs = np.array(list(zip(*paths)))
    payoffs = [f(STi,K,B) for STi in STs]
    return np.mean(payoffs)
    
def asianCall(S0,r,sigma,T,K,B=None):
    return asianPrice(S0,r,sigma,T,K,asianBarrierCallPayoff,B)

def asianPut(S0,r,sigma,T,K,B=None):
    return asianPrice(S0,r,sigma,T,K,asianBarrierPutPayoff,B)

In [374]:
ac = asianCall(40, 0.02, 0.3, 0.5, 39.5, 41.5)
print(ac)

0.0


In [338]:
ap = asianPut(40, 0.02, 0.3, 0.5, 41.5)
print(ap)

2.73055341231


In [342]:
def barrierPrice(S0,r,sigma,T,K,KOBarrier,f):
    paths=geomBrownIter(S0, r, sigma, T)
    STs = np.array(list(zip(*paths)))
    
    payoffs=[]
    for STi in STs:
        for Si in STi:
            if Si > KOBarrier:
                payoffs.append(0.0)
                break
        payoffs.append(f(Si,K))
    return np.mean(payoffs)*math.exp(-r*T)

def barrierCall(S0,r,sigma,T,K,KOBarrier):
    return barrierPrice(S0,r,sigma,T,K,KOBarrier,callPayoff)

def barrierPut(S0,r,sigma,T,K,KOBarrier):
    return barrierPrice(S0,r,sigma,T,K,KOBarrier,putPayoff)

In [343]:
bc = barrierCall(40, 0.02, 0.3, 0.5, 41.5, 40.5)
print(bc)

0.00606161345715


In [340]:
bp = barrierPut(40, 0.02, 0.3, 0.5, 41.5, 40.5)
print(bp)

0.739503336713
